<a href="https://colab.research.google.com/github/Takumi173/JPMA2022TF1-1/blob/main/JPMA_2022_TF_1_1_demo_(a)%E3%82%B3%E3%82%B5%E3%82%A4%E3%83%B3%E9%A1%9E%E4%BC%BC%E5%BA%A6%E3%81%AB%E3%82%88%E3%82%8B%E6%AF%94%E8%BC%83.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 前準備

## 必要なライブラリのインストール

In [1]:
!pip install mecab-python3 fugashi
!pip install jaconv neologdn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 577 kB 5.0 MB/s 
     |████████████████████████████████| 615 kB 52.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 57 kB 2.1 MB/s 
  Created wheel for jaconv: filename=jaconv-0.3.1-py3-none-any.whl size=16031 sha256=7824339b6396038974fc45ef04c9a5b50950157edfcd7269791ba35a7a3b45f4
  Stored in directory: /root/.cache/pip/wheels/42/96/fc/051c087a100ee53711dd37ac48a59e6832f620e4e65f650a66
  Created wheel for neologdn: filename=neologdn-0.5.1-cp38-cp38-linux_x86_64.whl size=178324 sha256=df5f9ac9aa802f5a797ff4caff42525811197030cda89857ac7fd146f0830758
  Stored in directory: /root/.cache/pip/wheels/3e/db/10/b3b26caa63c5da86ea3a25043cc4379a66bb3dd30d6f060a37
Successfully built jaconv neologdn


## データロードと環境構築

In [2]:
# MeCabとNEologdの設定
!apt install mecab libmecab-dev mecab-ipadic-utf8 file
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -a -y

# 環境変数でmecabrcの場所を指定
import os
os.environ['MECABRC'] = "/etc/mecabrc" 

# NEologdの展開場所を取得
import subprocess
cmd = 'echo `mecab-config --dicdir`"/mecab-ipadic-neologd"'
neologd_dic_dir_path = subprocess.check_output(cmd, shell=True).decode('utf-8').strip()

# 万病辞書のダウンロードと設定
!wget http://sociocom.jp/~data/2018-manbyo/data/MANBYO_201907_Dic-utf8.dic
manbyo_dic_path = 'MANBYO_201907_Dic-utf8.dic'

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libmagic-mgc libmagic1 libmecab2 mecab-ipadic mecab-jumandic
  mecab-jumandic-utf8 mecab-utils
The following NEW packages will be installed:
  file libmagic-mgc libmagic1 libmecab-dev libmecab2 mecab mecab-ipadic
  mecab-ipadic-utf8 mecab-jumandic mecab-jumandic-utf8 mecab-utils
0 upgraded, 11 newly installed, 0 to remove and 20 not upgraded.
Need to get 29.3 MB of archives.
After this operation, 282 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic-mgc amd64 1:5.32-2ubuntu0.4 [184 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic1 amd64 1:5.32-2ubuntu0.4 [68.6 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-up

# テキストの前処理

## 関数定義

In [3]:
import jaconv
import unicodedata
import neologdn
import re

import MeCab

# MeCab辞書にNEologdと万病辞書を指定
tagger = MeCab.Tagger("-d " + neologd_dic_dir_path + " -u " + manbyo_dic_path)

# MeCabによる分かち書き＆原形変換関数を定義
def genkei(text):
  words = []
  for c in tagger.parse(text).splitlines()[:-1]:
    # surfaceに単語、featureに解析結果を格納
    # 対象が存在しない場合は無視して次へ
    try:
        surface, feature = c.split('\t')
    except:
        continue

    # 解析結果から品詞と原形を取得
    hinshi = feature.split(',')[0]
    genkei = feature.split(',')[6]

    # 原形が定義されている場合は単語を原形に置き換え
    if feature.split(',')[-1].isdigit() == False and genkei != '*':
      surface = genkei

    # 残す品詞を定義
    if hinshi in ['名詞','形容詞','助動詞', '動詞'] and surface != '*':
      words.append(surface)
    else:
      continue

  return ' '.join(words)

# 文字の正規化後に上記関数を実行する事前処理関数を定義
def preprocess(text):
  text = jaconv.normalize(text, "NFKC")
  text = unicodedata.normalize("NFKC", text)
  text = neologdn.normalize(text)

  text = re.sub(r'。$', '', text)   # 文末の。を削除
  text = re.sub(r'\d+', '0', text)  # 連続する数値を0に置換

  text = genkei(text)
  return text

## 前処理の実行

In [4]:
import pandas as pd
import pickle

# 処理対象データをDataFrameに格納
Texts = pd.read_table('https://github.com/Takumi173/JPMA2022TF1-1/releases/download/20221226/Training.txt')

# 事前処理を実行したカラムを追加
Texts['wakati'] = Texts['TEXT'].apply(preprocess)

# 処理結果の確認
Texts

,LABEL1,LABEL2,TEXT,wakati
0,1,1,腎生検結果:光顕では軽度のメサンギウム細胞と基質の増加、尿細管間質はびまん性に高度の炎症細胞...,腎生検 結果 光顕 だ 軽度 メサンギウム 細胞 基質 増加 尿 細管 間質 びまん性 高度...
1,1,1,当院到着時は体温36.4°C、胸部違和感のみ残存していたが、採血でCRP 6.63 mg/d...,院 到着 時 体温 0 0 C 胸部違和感 残存 する いる た 採血 CRP 0 0 mg...
2,1,1,入院時の臨床検査では、重度のインスリン分泌能不良を認め、空腹時C-ペプチド免疫反応性(CPR...,入院 時 臨床検査 重度 インスリン分泌 能 不良 認める 空腹 時 C ペプチド 免疫反応...
3,1,1,中枢神経系に帰することのできる限局性または多発性所見として、2022/03/29に脳神経の単...,中枢神経 系 帰 する こと できる 限局 性 多発性 所見 0 0 0 脳神経 単一 複数...
4,1,1,36歳、女性、遺伝性血管浮腫3型のため、イカチバント酢酸塩(Icatibant)(シリンジ型...,0歳 女性 遺伝性 血管浮腫 0 型 ため イカチバント 酢酸塩 Icatibant シリン...
...,...,...,...,...
5014,0,0,減量,減量
5015,0,0,拒否,拒否
5016,0,0,休薬,休薬
5017,0,0,開始,開始


# 入力テキストとラベル付きデータのコサイン類似度計算

## 関数定義

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from tqdm.notebook import tqdm

# TF-IDFの計算関数
def CalcTfidf(TextSet):
  fn = TfidfVectorizer(token_pattern='\\S+')  # 半角スペースのみでSplitするように再定義
  vec = fn.fit_transform(TextSet)
  return vec.astype(float)

# ラベル付きデータに新規文章を加えてTF-IDFを計算する関数
def TfidfAll(LabelDF, NewTextList):

  PreDF = pd.DataFrame(NewTextList, columns = ["TEXT"])
  PreDF['wakati'] = PreDF['TEXT'].apply(preprocess)
  PreDF["LABEL1"] = 9

  ConcatDF = pd.concat([LabelDF,PreDF])
  Calctgt = ConcatDF['wakati'].tolist()

  ConcatDF["TFIDF"] = CalcTfidf(Calctgt).toarray().tolist()

  # Positive, Negative, 新規文章のDataFrameを返す
  NewDF = ConcatDF.query('LABEL1 == 9')
  PosDF = ConcatDF.query('LABEL1 == 1')
  NegDF = ConcatDF.query('LABEL1 == 0')

  return NewDF, PosDF, NegDF

# コサイン類似度を計算する関数
def cos_sim(t1, t2):
    return np.dot(t1, t2) / (np.linalg.norm(t1) * np.linalg.norm(t2))

# 比較対象の中で最大のコサイン類似度を返す関数
def max_cos_sim(Txt,TxtSet):
  x = 0
  id = 0
  for i, t in enumerate(TxtSet):
    if x < cos_sim(Txt, t):
      x = cos_sim(Txt, t)
      id = i
  return x, id

# NewDFをPosDFとNegDFに対してコサイン類似度の総当たりをする関数
def compareDF (NewDF, PosDF, NegDF):
  pos_sim = []
  neg_sim = []
  pt = []
  nt = []
  for t in tqdm(NewDF['TFIDF'], total = len(NewDF['TFIDF'])):
    pos, pid = max_cos_sim(np.array(t), np.array(PosDF['TFIDF']))
    pos_sim.append(pos)
    pt += [Pos.iat[pid,1]]

    neg, nid = max_cos_sim(np.array(t), np.array(NegDF['TFIDF']))
    neg_sim.append(neg)
    nt += [Neg.iat[nid,1]]

  # 最大のコサイン類似度とその時の比較対象テキストをそれぞれ格納する
  NewDF["Positive"] = pos_sim
  NewDF["Negative"] = neg_sim
  NewDF["Positive Text"] = pt
  NewDF["Negative Text"] = nt

  # 不要なカラムを削除
  NewDF = NewDF.drop(['LABEL1', 'LABEL2', 'TFIDF'], axis=1)

  # PositiveとNegativeの値からPredictionを算出
  NewDF["Pred"] = NewDF[["Positive","Negative"]].idxmax(axis=1).apply(lambda x: 0 if x == "Negative" else 1)
  return NewDF

## ラベル付きデータによる検証

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score

# 実行完了までに30-40分かかる
X_train, X_test, y_train, y_test = train_test_split(Texts, Texts["LABEL1"], test_size=0.15, random_state = 0)

New, Pos, Neg = TfidfAll(X_train, X_test)
Result = compareDF(New, Pos, Neg)
Result['Label'] = y_test

display(Result)

print('\n****: Scores')
print('        F1:', f1_score(Result["Label"], Result["Pred"]))
print('    Recall:', recall_score(Result["Label"], Result["Pred"]))
print('  Accuracy:', accuracy_score(Result["Label"], Result["Pred"]))
print(' Precision:', precision_score(Result["Label"], Result["Pred"]))

  0%|          | 0/753 [00:00<?, ?it/s]

,TEXT,wakati,Positive,Negative,Positive Text,Negative Text,Pred,Label
971,2022/01/27、SARS-CoV-2検査(PCR)で、COVID-19陽性を示した。,0 0 0 SARS CoV 0 検査 PCR だ COVID 0 陽性 示す た,0.690833,0.677969,0.0,0.0,1,1
3707,さらなる以上の追加情報は期待できない。,以上 追加 情報 期待 できる ない,0.191284,0.893960,0.0,0.0,0,0
2708,脳神経系、運動系、感覚系に異常はなく、四肢腱反射正常で病的反射は両側陰性だった。,脳神経 系 運動系 感覚 系 異常 ない 四肢腱反射正常 だ 病的 反射 両側 陰性 だ た,0.334253,0.381812,0.0,0.0,0,0
302,2021/12/17(ワクチン接種15日後)、症状の増悪があり入院となった。,0 0 0 ワクチン接種 0 日 後 症状 増悪 ある 入院 なる た,0.607021,0.621179,0.0,0.0,0,1
3805,患者の家族歴は特になしと報告された。,患者 家族 歴 ない 報告 する れる た,0.303564,0.836914,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...
517,両側の肺炎と呼吸不全を指摘され入院。,両側 肺炎 呼吸不全 指摘 する れる 入院,0.428584,0.267076,1.0,0.0,1,1
3815,その他の画像検査は実施されなかった。,その他 画像 検査 実施 する れる ない た,0.370027,0.892800,0.0,0.0,0,0
2958,2022/05/24、事象の転帰は軽快であった(報告の通り)。,0 0 0 事象 転帰 軽快 だ ある た 報告 通り,0.552513,0.931410,0.0,0.0,0,0
3741,2022/03/07症状軽快し、退院。,0 0 0 症状 軽快 する 退院,0.538200,0.936323,1.0,0.0,0,0



****: Scores
        F1: 0.8353741496598639
    Recall: 0.8186666666666667
  Accuracy: 0.8393094289508632
 Precision: 0.8527777777777777


## 入力テキストの予測

In [10]:
txt = [
  "組織脳脊髄液検査では異常が無かったが、細菌培養及びウイルス検査は異常を認めた。",
  "2022/05/23、倦怠感(障害)発現、転帰「未回復」。",
  "日付不明発熱。",
  "発現2022/04/08、心電図ST部分上昇(医学的に重要)、転帰「軽快」、「心電図ST上昇」と記述された。",
  "膀胱ポリープ(入院)、転帰「不明」。",
  "血中クレアチンホスホキナーゼ増加(入院)、発現2022/01/15 05:00、転帰「軽快」、「血液検査でCK上昇」と記載された。",
  "小腸および皮膚の病理組織を提出した。",
  "17:30頃から徐々に症状は回復した。",
  "冠攣縮性狭心症が疑われ、冠動脈造影によるアセチルコリン負荷試験を施行したが有意な所見は認めなかった。",
  "昨日尿管結石が見つかり、下記薬を服用中であった。",
  "血漿交換療法を実施して徐々に症状の改善を得たが、発症後よりはADL軽度低下した状態。",
  "新規感染であった。",
  "日中持続したが、夜就寝可能であった。",
  "患者は食欲があった。",
  "患者には、20年の喫煙歴と30年の禁煙歴があった。",
  ]

New, Pos, Neg = TfidfAll(Texts,txt)
Result = compareDF(New, Pos, Neg)

display(Result)

  0%|          | 0/15 [00:00<?, ?it/s]

,TEXT,wakati,Positive,Negative,Positive Text,Negative Text,Pred
0,組織脳脊髄液検査では異常が無かったが、細菌培養及びウイルス検査は異常を認めた。,組織 脳脊髄液 検査 異常 無い た 細菌 培養 ウイルス検査 異常 認める た,0.458811,0.583790,0.0,0.0,0
1,2022/05/23、倦怠感(障害)発現、転帰「未回復」。,0 0 0 倦怠感 障害 発現 転帰 回復,0.676699,0.487364,1.0,0.0,1
2,日付不明発熱。,日付 不明発熱,0.248478,0.298087,0.0,0.0,0
3,発現2022/04/08、心電図ST部分上昇(医学的に重要)、転帰「軽快」、「心電図ST上昇...,発現 0 0 0 心電図 ST 部分 上昇 医学 的 重要 転帰 軽快 心電図 ST 上昇 ...,0.627623,0.431186,0.0,0.0,1
4,膀胱ポリープ(入院)、転帰「不明」。,膀胱ポリープ 入院 転帰 不明,0.496565,0.318196,1.0,0.0,1
5,血中クレアチンホスホキナーゼ増加(入院)、発現2022/01/15 05:00、転帰「軽快」...,血 中 クレアチンホスホキナーゼ 増加 入院 発現 0 0 0 0 0 転帰 軽快 血液検査...,0.550512,0.477855,1.0,0.0,1
6,小腸および皮膚の病理組織を提出した。,小腸 皮膚 病理 組織 提出 する た,0.333843,0.356783,0.0,0.0,0
7,17:30頃から徐々に症状は回復した。,0 0 頃 症状 回復 する た,0.641342,0.782016,0.0,0.0,0
8,冠攣縮性狭心症が疑われ、冠動脈造影によるアセチルコリン負荷試験を施行したが有意な所見は認めな...,冠 攣縮性 狭心症 疑う れる 冠動脈造影 アセチルコリン 負荷 試験 施行 する た 有意...,0.414911,0.367236,0.0,0.0,1
9,昨日尿管結石が見つかり、下記薬を服用中であった。,昨日 尿管結石 見つかる 下記 薬 服用 中 だ ある た,0.173670,0.289233,1.0,0.0,0
